In [160]:
import pandas as pd
import shapefile
import matplotlib.pyplot as plt
import geopandas as gpd
import pysal as ps
import pylab as plot
import numpy as np
import pickle
import geocoder
import string

from shapely.geometry import shape


%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [161]:
gdf_medicare = pd.read_pickle('gdf_medicare')

In [162]:
gdf_medicare.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,long_lat,geometry
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,"$32,963.07","$5,777.24","$4,763.73","(-85.3928604125977, 31.2167854309082)",POINT (-85.3928604125977 31.2167854309082)
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,"$15,131.85","$5,787.57","$4,976.71","(-86.3727798461914, 34.1399955749512)",POINT (-86.37277984619141 34.1399955749512)
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,"$37,560.37","$5,434.95","$4,453.79","(-87.6751022338867, 34.8017730712891)",POINT (-87.6751022338867 34.8017730712891)
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,"$13,998.28","$5,417.56","$4,129.16","(-86.636100769043, 33.6505241394043)",POINT (-86.636100769043 33.6505241394043)
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,"$31,633.27","$5,658.33","$4,851.44","(-86.8290939331055, 33.2154388427734)",POINT (-86.8290939331055 33.2154388427734)


In [163]:
gdf_medicare['city_state'] = gdf_medicare['Provider City'] + ', ' + gdf_medicare['Provider State']

In [164]:
gdf_hrr = pd.read_pickle('Dataframes/gdf_hrr')

In [165]:
gdf_hrr.head()

,HRRCITY,HRRNUM,HRR_BDRY_I,geometry
0,AL- BIRMINGHAM,1,1,"(POLYGON ((-85.89658 32.70483, -85.90771000000..."
1,AL- DOTHAN,2,2,"(POLYGON ((-86.194 31.43991, -86.1947800000000..."
2,AL- HUNTSVILLE,5,3,"(POLYGON ((-86.69474000000001 34.42309, -86.71..."
3,AL- MOBILE,6,4,"(POLYGON ((-87.55554000000001 30.47282, -87.56..."
4,AL- MONTGOMERY,7,5,"POLYGON ((-86.49925 31.52517, -86.642280000000..."


In [166]:
gdf_medicare.columns

Index(['DRG Definition', 'Provider Id', 'Provider Name',
       'Provider Street Address', 'Provider City', 'Provider State',
       'Provider Zip Code', 'Hospital Referral Region Description',
       ' Total Discharges ', ' Average Covered Charges ',
       ' Average Total Payments ', 'Average Medicare Payments', 'long_lat',
       'geometry', 'city_state'],
      dtype='object')

In [167]:
gdf_medicare.rename(columns={' Total Discharges ': 'Total Discharges', \
                         ' Average Covered Charges ': 'Average Covered Charges',\
                  ' Average Total Payments ' : 'Average Total Payments' }, inplace=True)

In [168]:
numerical_col = ['Total Discharges', 'Average Covered Charges', 'Average Total Payments', 'Average Medicare Payments']

In [169]:
gdf_medicare[numerical_col] = \
gdf_medicare[numerical_col].apply(lambda x: x.str.replace('[\$,]', ''))

In [170]:
gdf_medicare[numerical_col] = \
gdf_medicare[numerical_col].apply(lambda x: x.astype(float))

In [171]:
numerical_dict = dict.fromkeys(numerical_col, 'mean')

In [172]:
numerical_dict

{'Average Covered Charges': 'mean',
 'Average Medicare Payments': 'mean',
 'Average Total Payments': 'mean',
 'Total Discharges': 'mean'}

In [173]:
df_mean = gdf_medicare.groupby(['Hospital 1Referral Region Description']\
                                            ,as_index=False).agg(numerical_dict)

In [174]:
df_mean['Hospital Referral Region Description'] = \
df_mean['Hospital Referral Region Description'].apply(lambda x: x.replace(' ', ''))

In [175]:
df_mean['Hospital Referral Region Description'] = \
df_mean['Hospital Referral Region Description'].apply(lambda x: x.upper())

In [176]:
gdf_hrr['HRRCITY_cleaned'] = gdf_hrr['HRRCITY']

gdf_hrr['HRRCITY_cleaned'] = \
gdf_hrr['HRRCITY_cleaned'].apply(lambda x: x.replace(' ', ''))

gdf_hrr['HRRCITY_cleaned'] = \
gdf_hrr['HRRCITY_cleaned'].apply(lambda x: x.upper())

In [177]:
gdf_hrr['HRRCITY_cleaned'] = \
gdf_hrr['HRRCITY_cleaned'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))

df_mean['Hospital Referral Region Description'] = \
df_mean['Hospital Referral Region Description'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))

In [180]:
hrr_na_update = {'CAALAMEDACOUNTY': 'CAALAMEDACO', 'CACONTRACOSTACOUNTY' : 'CACONTRACOSTACO', \
'CAORANGECOUNTY' : 'CAORANGECO', 'CAPALMSPRINGSRANCHOM' : 'CAPALMSPRRANCHOMIR'\
,'CASANMATEOCOUNTY' : 'CASANMATEOCO', 'NYMANHATTAN': 'NYNEWYORK'}

In [181]:
df_mean["Hospital Referral Region Description"].replace(hrr_na_update, inplace=True)

In [182]:
merged_hrr_gdf = pd.merge(gdf_hrr, df_mean, left_on = 'HRRCITY_cleaned', right_on = 'Hospital Referral Region Description'\
        ,how = 'outer')

In [183]:
merged_hrr_gdf

,HRRCITY,HRRNUM,HRR_BDRY_I,geometry,HRRCITY_cleaned,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,AL- BIRMINGHAM,1,1,"(POLYGON ((-85.89658 32.70483, -85.90771000000...",ALBIRMINGHAM,ALBIRMINGHAM,36.255715,35875.130728,7724.118841,6577.098107
1,AL- DOTHAN,2,2,"(POLYGON ((-86.194 31.43991, -86.1947800000000...",ALDOTHAN,ALDOTHAN,38.452941,27610.974000,7038.365971,6012.843618
2,AL- HUNTSVILLE,5,3,"(POLYGON ((-86.69474000000001 34.42309, -86.71...",ALHUNTSVILLE,ALHUNTSVILLE,51.631450,28310.887690,7517.767617,6194.050393
3,AL- MOBILE,6,4,"(POLYGON ((-87.55554000000001 30.47282, -87.56...",ALMOBILE,ALMOBILE,41.545290,28951.019982,7639.114946,6356.235942
4,AL- MONTGOMERY,7,5,"POLYGON ((-86.49925 31.52517, -86.642280000000...",ALMONTGOMERY,ALMONTGOMERY,33.720548,25152.752548,7620.635288,6572.985890
5,AL- TUSCALOOSA,9,6,"POLYGON ((-87.31229 33.08598, -87.308500000000...",ALTUSCALOOSA,ALTUSCALOOSA,55.796512,26422.386919,8352.626628,7292.552209
6,AK-Anchorage,10,7,"(POLYGON ((-105.627887 29.245417, -105.627887 ...",AKANCHORAGE,AKANCHORAGE,26.588745,40348.743333,14572.391732,12958.969437
7,AZ- MESA,11,8,"POLYGON ((-111.21145 33.08493, -111.21638 33.0...",AZMESA,AZMESA,41.142276,46567.402297,9775.701687,8317.502866
8,AZ- PHOENIX,12,9,"(POLYGON ((-111.5151 33.01583, -111.51156 33.0...",AZPHOENIX,AZPHOENIX,33.595128,39288.264806,10629.743436,9213.646886
9,AZ- SUN CITY,14,10,"POLYGON ((-112.40912 33.63857, -112.4542 33.63...",AZSUNCITY,AZSUNCITY,59.544503,43732.348796,8589.015288,7549.713141


In [184]:
merged_hrr_gdf[merged_hrr_gdf.isnull().any(axis=1)]

,HRRCITY,HRRNUM,HRR_BDRY_I,geometry,HRRCITY_cleaned,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments


In [187]:
merged_hrr_gdf.to_pickle('Dataframes/merged_hrr_gdf')

In [193]:
gdf_medicare['HRRD'] = gdf_medicare['Hospital Referral Region Description']

In [194]:
gdf_medicare['HRRD'] = \
gdf_medicare['HRRD'].apply(lambda x: x.replace(' ', ''))

In [195]:
gdf_medicare['HRRD'] = \
gdf_medicare['HRRD'].apply(lambda x: x.upper())

In [196]:
gdf_medicare['HRRD'] = \
gdf_medicare['HRRD'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))

In [197]:
gdf_medicare["HRRD"].replace(hrr_na_update, inplace=True)

In [198]:
gdf_medicare

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments,long_lat,geometry,city_state,HRRD
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91.0,32963.07,5777.24,4763.73,"(-85.3928604125977, 31.2167854309082)",POINT (-85.3928604125977 31.2167854309082),"DOTHAN, AL",ALDOTHAN
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14.0,15131.85,5787.57,4976.71,"(-86.3727798461914, 34.1399955749512)",POINT (-86.37277984619141 34.1399955749512),"BOAZ, AL",ALBIRMINGHAM
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24.0,37560.37,5434.95,4453.79,"(-87.6751022338867, 34.8017730712891)",POINT (-87.6751022338867 34.8017730712891),"FLORENCE, AL",ALBIRMINGHAM
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25.0,13998.28,5417.56,4129.16,"(-86.636100769043, 33.6505241394043)",POINT (-86.636100769043 33.6505241394043),"BIRMINGHAM, AL",ALBIRMINGHAM
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18.0,31633.27,5658.33,4851.44,"(-86.8290939331055, 33.2154388427734)",POINT (-86.8290939331055 33.2154388427734),"ALABASTER, AL",ALBIRMINGHAM
5,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10023,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116,AL - Montgomery,67.0,16920.79,6653.80,5374.14,"(-86.2304229736328, 32.3287010192871)",POINT (-86.2304229736328 32.3287010192871),"MONTGOMERY, AL",ALMONTGOMERY
6,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10029,EAST ALABAMA MEDICAL CENTER AND SNF,2000 PEPPERELL PARKWAY,OPELIKA,AL,36801,AL - Birmingham,51.0,11977.13,5834.74,4761.41,"(-85.3766860961914, 32.6516761779785)",POINT (-85.37668609619141 32.6516761779785),"OPELIKA, AL",ALBIRMINGHAM
7,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10033,UNIVERSITY OF ALABAMA HOSPITAL,619 SOUTH 19TH STREET,BIRMINGHAM,AL,35233,AL - Birmingham,32.0,35841.09,8031.12,5858.50,"(-86.7937774658203, 33.5106315612793)",POINT (-86.7937774658203 33.5106315612793),"BIRMINGHAM, AL",ALBIRMINGHAM
8,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10039,HUNTSVILLE HOSPITAL,101 SIVLEY RD,HUNTSVILLE,AL,35801,AL - Huntsville,135.0,28523.39,6113.38,5228.40,"(-86.5851287841797, 34.7274475097656)",POINT (-86.5851287841797 34.7274475097656),"HUNTSVILLE, AL",ALHUNTSVILLE
9,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10040,GADSDEN REGIONAL MEDICAL CENTER,1007 GOODYEAR AVENUE,GADSDEN,AL,35903,AL - Birmingham,34.0,75233.38,5541.05,4386.94,"(-85.7613830566406, 34.0703048706055)",POINT (-85.7613830566406 34.0703048706055),"GADSDEN, AL",ALBIRMINGHAM


In [199]:
gdf_medicare.to_pickle('Dataframes/gdf_medicare_correct_hrr')